In [1]:
import numpy as np

from matplotlib import pyplot as plt 

import glob
import pandas as pd
import plotly.graph_objects as go

import plotly.io as pio
import plotly.express as px
import plotly

In [2]:
place_mapping = {'137f63':'door', '680415': 'front corner', '6f7f19':'back corner/exhaust',
                    '110e60':'back', '620512': 'front-desk', '6e0d1e': 'front-tripod'}

sensor_mapping = {'BC5F2E8BD0E9B21D':'680415', 'DA8590A0DD5EC12B':'6f7f19', 
                    '1837FE6A6AD70C29':'620512', '124BF175470BC117':'137f63', '033EBECC09FAA564':'110e60', '38869427C2B44E7E':'6e0d1e'}

In [3]:
# x_mapping  = {k:v for k,v  in zip(sensor_mapping.keys(), [.65, .9, .95, 0, 0, .9 ]) }
# y_mapping  = {k:v for k,v in zip(sensor_mapping.keys(), [.45, .45, 0, 0, .85, 1])  }
# z_mapping  = {k:v for k,v in zip(sensor_mapping.keys(), [.3, .5, 0.85, 0.9, .2, .6]) }

In [4]:
sensor_ids = ['1837FE6A6AD70C29', '38869427C2B44E7E', '124BF175470BC117', '033EBECC09FAA564', 'DA8590A0DD5EC12B', 'BC5F2E8BD0E9B21D']

x_mapping  = {k:v for k,v  in zip(sensor_ids, [.65, .9, .95, 0, 0, .9 ]) }
y_mapping  = {k:v for k,v in zip(sensor_ids, [.45, .45, 0, 0.1, .85, 1])  }
z_mapping  = {k:v for k,v in zip(sensor_ids, [.3, .5, 0.9, 0.8, .2, .6]) }

In [5]:
loc_mapping = {sid:place_mapping[sensor_mapping[sid]] for sid in sensor_ids}
loc_mapping

{'1837FE6A6AD70C29': 'front-desk',
 '38869427C2B44E7E': 'front-tripod',
 '124BF175470BC117': 'door',
 '033EBECC09FAA564': 'back',
 'DA8590A0DD5EC12B': 'back corner/exhaust',
 'BC5F2E8BD0E9B21D': 'front corner'}

In [6]:
col_names =['Epoch_UTC', 'Local_Date_Time', 'MassConc_1p0',
       'MassConc_2p5',
       'MassConc_4p0',
       'MassConc_10p',
       'NumbConc_0p5',
       'NumbConc_1p0',
       'NumbConc_2p5',
       'NumbConc_4p0',
       'NumbConc_10p',
       'TypPartSize', 'id', 'x', 'y', 'z', 'loc']

In [18]:
import glob
file_paths = [f for f in glob.glob('../../data/pm_data/feb27-low/*.edf')]

data_frames = []

start_ts = 50
end_ts =250

for i, _file in enumerate(file_paths):
    print(_file)
   # df = pd.read_csv(_file, sep="	", parse_dates=[1], on_bad_lines='skip')
    #df = pd.read_csv(_file, sep="	 ", parse_dates=[1], error_bad_lines='skip')
    df = pd.read_csv(_file, sep="	", parse_dates=[1], comment = '#')
    df['Local_Date_Time'] = df['Local_Date_Time'].dt.floor('S').dt.tz_localize(None)

    sensor_id = _file.split("_")[-1].split(".")[0]
    df['id'] = sensor_id
    print(sensor_id, x_mapping[sensor_id])
    df['x'] = x_mapping[sensor_id]
    df['y'] = y_mapping[sensor_id]
    df['z'] = z_mapping[sensor_id]
    df['loc'] = loc_mapping[sensor_id]
    df.columns = col_names
    df = df.sort_values(by=['Epoch_UTC'])
    df = df.reset_index()
    df['time_elapsed'] = df.index
    df = df.dropna()
    data_frames.append(df.iloc[start_ts:end_ts, :])
    # print(data_frames[i].iloc[10])

../../data/pm_data/feb27-low/2023-02-27_14-59-52-SPS3x_DA8590A0DD5EC12B.edf
DA8590A0DD5EC12B 0
../../data/pm_data/feb27-low/2023-02-27_14-59-52-SPS3x_033EBECC09FAA564.edf
033EBECC09FAA564 0
../../data/pm_data/feb27-low/2023-02-27_14-59-52-SPS3x_38869427C2B44E7E.edf
38869427C2B44E7E 0.9
../../data/pm_data/feb27-low/2023-02-27_14-59-52-SPS3x_124BF175470BC117.edf
124BF175470BC117 0.95
../../data/pm_data/feb27-low/2023-02-27_14-59-52-SPS3x_BC5F2E8BD0E9B21D.edf
BC5F2E8BD0E9B21D 0.9
../../data/pm_data/feb27-low/2023-02-27_14-59-52-SPS3x_1837FE6A6AD70C29.edf
1837FE6A6AD70C29 0.65


In [19]:
combined_df = pd.concat(data_frames, axis=0)

In [20]:
plot_col = 'MassConc_2p5'
min_conc_val = combined_df[plot_col].min()
max_conc_val = combined_df[plot_col].max()
fig = px.scatter_3d(combined_df,x='x', y='y', z='z', text='loc', color= 'MassConc_2p5', size='MassConc_2p5', animation_frame='time_elapsed', size_max=50, range_color=[min_conc_val, max_conc_val])
# fig.update_layout(title_text='Sensors Location in a Room', title_x= 0.43, width= 750, height = 400)
fig.update_layout(
    scene = dict(
        xaxis = dict(nticks=5, range=[0,1],),
                     yaxis = dict(nticks=5, range=[0,1],),
                     zaxis = dict(nticks=5, range=[0,1],),
                     aspectratio=dict(x=1, y=1, z=1),
                ),
        )


fig.show()


In [21]:
import plotly.io as pio
pio.write_html(fig, file='figure.html', auto_open=True)